In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
import pybedtools
from scipy.stats import norm
import seaborn as sns
pybedtools.helpers.set_tempdir('/data/projects/temp')

In [2]:
attention_score_path = "/data/projects/Enhancer/DNABERT_Output/Visualization_Enhancer_Encode/Predictions/atten_score200.npy"
prediction_path = "/data/projects/Enhancer/DNABERT_Output/Visualization_Enhancer_Encode/Predictions/pred_results.npy"
#data_path = "/data/projects/Enhancer/Finetuned_data/Consensus_filter_1/Visualization/dev.tsv"
raw_data_path = "/data/projects/Enhancer/Finetuned_data/Consensus_filter_1/RF_dev.tsv"
bed_folder_path= "/data/projects/Resources/ENCODE_data/ChIP-seq_data/ChIP-seq_peaks_bed"
chi_seq_path= "/data/projects/Enhancer/ChipSeq_Intersection/ChIP-seq_peaks.csv"

In [3]:
attention_data = np.load(attention_score_path)
pred_result = np.load(prediction_path)
#data = pd.read_csv(data_path, sep="\t") 
raw_data = pd.read_csv(raw_data_path, sep="\t")
intersected_chip_data = pd.read_csv(chi_seq_path, sep=",")

In [4]:
intersected_chip_data

,peak_chr,peak_start,peak_end,target,biosample,biosample_name,enhancer_chr,enhancer_start,enhancer_end,#overlap,coord
0,chr13,23651920,23652200,EZH2phosphoT487,cell line,Loucy,chr13,23652031,23652230,169,chr13:23652031-23652230
1,chr13,23652027,23652418,H3K27ac,cell line,HepG2,chr13,23652031,23652230,199,chr13:23652031-23652230
2,chr13,23652041,23653601,H3K27ac,in vitro differentiated cells,smooth muscle cell,chr13,23652031,23652230,189,chr13:23652031-23652230
3,chr13,23652101,23652837,H3K27me3,cell line,HeLa-S3,chr13,23652031,23652230,129,chr13:23652031-23652230
4,chr13,23652007,23653025,H3K27me3,cell line,OCI-LY3,chr13,23652031,23652230,199,chr13:23652031-23652230
...,...,...,...,...,...,...,...,...,...,...,...
57638,chr15,41719929,41720547,H3K79me2,cell line,Karpas-422,chr15,41720338,41720537,199,chr15:41720338-41720537
57639,chr15,41720160,41720894,H3K79me2,cell line,SK-N-SH,chr15,41720338,41720537,199,chr15:41720338-41720537
57640,chr15,41718815,41720465,H3K79me2,in vitro differentiated cells,bipolar neuron,chr15,41720338,41720537,127,chr15:41720338-41720537
57641,chr15,41720369,41720983,H3K79me2,in vitro differentiated cells,smooth muscle cell,chr15,41720338,41720537,168,chr15:41720338-41720537


In [5]:
pred_result

array([0.80196536, 0.00386043, 0.1981992 , ..., 0.21390572, 0.39133212,
       0.8598932 ], dtype=float32)

In [6]:
attention_data

array([[0.15736438, 0.12613323, 0.12205527, ..., 0.08286749, 0.09156235,
        0.1009498 ],
       [0.0706074 , 0.06797632, 0.07054477, ..., 0.04955077, 0.04924984,
        0.05282408],
       [0.12788138, 0.13338041, 0.12745156, ..., 0.05847935, 0.06300342,
        0.08041372],
       ...,
       [0.13845803, 0.12782378, 0.12814937, ..., 0.08183175, 0.08405915,
        0.07355097],
       [0.06446469, 0.09207337, 0.08236827, ..., 0.0796929 , 0.08804073,
        0.11264496],
       [0.0781696 , 0.08220336, 0.08146374, ..., 0.06815664, 0.07253033,
        0.09001699]])

In [7]:
true_data = raw_data[['coord',"sequence","label"]]
true_data

,coord,sequence,label
0,chr13:23652031-23652230,TCCTTTCTGCACAGGGTTATTACAGCGCCCAGAAAGTGGGTATGTA...,1
1,chr14:17061576-17061775,TCAAATAAAAACTAGACAGAAGCATTCTCATAAGCTTGTTTGTGAT...,0
2,chr9:24999931-25000130,CCACCAAGGAGACCTCATTTGTAGATTTGGCAAAGTATGGCACAAT...,0
3,chr18:52169445-52169644,CATTTAAATTTTAAATGTATTTAACATTTAGAGAATAGTATAAATA...,0
4,chr9:32366053-32366252,AAGTTGAAATTAAATAAATCCTAAATTTGAATCAGGTGTGTTGGTA...,0
...,...,...,...
3186,chr9:38725772-38725971,CCCACCACCACGCCTGGCTAATTTTTCTGTATTTTCAGTAGAGATG...,1
3187,chr1:105315643-105315842,TATTTTATTTTTACTGTTTTGTCCCACATGAGCTTGCTTATATATT...,0
3188,chr7:27950006-27950205,GAGATAACTTACCTGCTTTTCACGATACTTTTGATACAATCTCATT...,1
3189,chr2:171047833-171048032,TCATCTTCTAACCACTCTGGATTATATCACCAATTATAAACGGCTT...,1


In [8]:
categorical_predictions = (pred_result > 0.5).astype(int)
true_data['Prediction_result'] = categorical_predictions 
true_data

df_visualize = true_data[(true_data['label'] == 1) & (true_data['Prediction_result'] == 1)]
# df_visualize_all = df_visualize.merge(true_data, left_index=True, right_index=True, how='inner')
df_visualize

/data/projects/temp/ipykernel_536879/3111908572.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  true_data['Prediction_result'] = categorical_predictions


,coord,sequence,label,Prediction_result
0,chr13:23652031-23652230,TCCTTTCTGCACAGGGTTATTACAGCGCCCAGAAAGTGGGTATGTA...,1,1
11,chr10:45519836-45520035,GAGGTGGAATCAGTAAAACTTATTGATGAGTGGCGCATTAAGAGCA...,1,1
12,chr3:193057772-193057971,TAGGTTTTACAATTCCTTTCAACACCAAATTAAGATCAGAAGGTAT...,1,1
13,chr3:77507468-77507667,TCTTGCACACAGAAGACTGCAAGAATGGGCTCTGCCTATCTAAAAA...,1,1
18,chr1:187980069-187980268,GCACTGTGGCTCATGCCTGTAATCCCAGCACTTTGGGAGGCTGAGG...,1,1
...,...,...,...,...
3181,chr5:64261727-64261926,ATATCAGGTGGAAAGAAACAAAGCTGCTCAAGTGAGAACAACCAAA...,1,1
3183,chr12:43813667-43813866,TCTACACTCTCAAAGAAGCTCATTTCAAGGCCCTTCCTTACTCAGT...,1,1
3184,chr1:204552002-204552201,GTGGACCGGGCGTAGTGGCTCACGCCTGTAATCCCAACACTTTGGG...,1,1
3186,chr9:38725772-38725971,CCCACCACCACGCCTGGCTAATTTTTCTGTATTTTCAGTAGAGATG...,1,1


In [9]:
positive_attention_score = attention_data[df_visualize.index.values]
positive_attention_score

array([[0.15736438, 0.12613323, 0.12205527, ..., 0.08286749, 0.09156235,
        0.1009498 ],
       [0.14155952, 0.11704731, 0.10161122, ..., 0.08688743, 0.08341995,
        0.08914666],
       [0.1592955 , 0.13399013, 0.1245085 , ..., 0.06538571, 0.06437823,
        0.0754427 ],
       ...,
       [0.09295924, 0.08530137, 0.08252531, ..., 0.05208671, 0.04840521,
        0.05569853],
       [0.08796743, 0.09712145, 0.10442137, ..., 0.0893162 , 0.10288811,
        0.11376228],
       [0.0781696 , 0.08220336, 0.08146374, ..., 0.06815664, 0.07253033,
        0.09001699]])

In [27]:
chipseq_attention_path = os.path.dirname(attention_score_path)+"/Visualization/ChipSeq_Attention_Intersection"
if not os.path.exists(chipseq_attention_path):
    os.makedirs(chipseq_attention_path)

In [43]:
for index, row in enumerate(positive_attention_score):
    plt.figure(figsize=(12, 9))
    title = df_visualize['coord'].iloc[index]
    if((index%100)==0):
        print(index , title)
    enhancer_start, enhancer_end = map(int, title.split(":")[1].split("-"))
    #print(title)
    enhancer_x_points = np.arange(enhancer_start, enhancer_end+1)
    enhancer_y_points = row
    max_height = max(row)
   
    
    # Generate x and y values
    x_vals = []
    y_vals = []
    combined_labels = []
    x_vals.extend(enhancer_x_points)
    y_vals.extend(enhancer_y_points)
    combined_labels.extend(["Attention_score"] * len(x_vals))
    df_temp= intersected_chip_data[intersected_chip_data['coord']==title]
    #df_temp =df_temp.head(5)
    #print(max_height)

    for i, row in df_temp.iterrows():
        x_range = np.linspace(row['peak_start'], row['peak_end'], 100)  # Create 100 points for smoothness
        midpoint = (row['peak_end'] + row['peak_start']) / 2
        half_width = (row['peak_end'] - row['peak_start']) / 2

        # Generate bell-shaped y-values using sine function
        y_range = np.sin(np.pi * (x_range - row['peak_start']) / (2 * half_width))*max_height

        x_vals.extend(x_range)
        y_vals.extend(y_range)
        combined_labels.extend([row['target']+"__"+row['biosample_name']] * len(x_range))

    plot_df = pd.DataFrame({
        'x': x_vals,
        'y': y_vals,
        'combined_label': combined_labels
    })
    
    # Set default color palette
    palette = sns.color_palette("tab10", n_colors=len(plot_df['combined_label'].unique()))

    # Plot
    fig, ax = plt.subplots(figsize=(20, 6))
    for idx, label in enumerate(plot_df['combined_label'].unique()):
        subset = plot_df[plot_df['combined_label'] == label]
        color = palette[idx]
        ax.plot(subset['x'], subset['y'], label=label, color=color)
        alpha_value = 0.8 if label == "Attention_score" else 0.1
        ax.fill_between(subset['x'], subset['y'], color=color, alpha=alpha_value)  # Fill the area with a transparent color of the line color

    # Format x-axis ticks to display the entire number
    # Format x-axis ticks to display the entire number and focus on the enhancer region
    margin = 0
    ax.set_xlim(enhancer_start - margin, enhancer_end + margin)
    ax.get_xaxis().get_major_formatter().set_scientific(False)
    ax.get_xaxis().get_major_formatter().set_useOffset(False)

    #ax.set_ylim(0, 0.13+0.20)
    # Position the legend in the top right corner of the plot
    ax.legend(title='Labels', loc='upper left', bbox_to_anchor=(1, 1))
    plt.title(f"Intersection of the Enhancer region [{title}] with various ENCODE ChIP-seq regions.")
    plt.xlabel("Chromosome Coordinates")
    plt.ylabel("Attention Value")
    #plt.show()
    plt.savefig(chipseq_attention_path+"/"+f"{title}.png", bbox_inches='tight')
    plt.close()

0 chr13:23652031-23652230


/data/projects/temp/ipykernel_536879/1347741400.py:46: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(figsize=(20, 6))
/data/projects/temp/ipykernel_536879/1347741400.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(12, 9))


100 chr9:20394160-20394359
200 chr19:56379952-56380151
300 chr4:57026631-57026830
400 chr7:99260816-99261015
500 chr21:25709763-25709962
600 chr12:58611503-58611702
700 chr16:30296424-30296623
800 chr14:31168752-31168951
900 chr9:22406052-22406251
1000 chr9:98067700-98067899
1100 chr12:72832475-72832674


<Figure size 1200x900 with 0 Axes>

<Figure size 1200x900 with 0 Axes>

...

<Figure size 1200x900 with 0 Axes>

<Figure size 1200x900 with 0 Axes>

<Figure size 1200x900 with 0 Axes>

<Figure size 1200x900 with 0 Axes>

<Figure size 1200x900 with 0 Axes>